<a href="https://colab.research.google.com/github/NuttakitDW/kbtg_data/blob/master/Clean_M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import datetime

In [0]:
cc = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/cc.csv')
dmg = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/demographics.csv')
kplus = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/kplus.csv')
train = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/test.csv')

In [215]:
cc.head()

,cc_no,pos_dt,cc_txn_amt
0,37069,2018-05-10,5000
1,37069,2018-06-04,12000
2,37069,2018-04-03,5000
3,37069,2018-04-22,1600
4,37069,2018-01-21,5000


In [216]:
cc['month'] = pd.DatetimeIndex(cc['pos_dt']).month
cc.head()

,cc_no,pos_dt,cc_txn_amt,month
0,37069,2018-05-10,5000,5
1,37069,2018-06-04,12000,6
2,37069,2018-04-03,5000,4
3,37069,2018-04-22,1600,4
4,37069,2018-01-21,5000,1


In [217]:
cc = cc.groupby(['cc_no', 'month'])['cc_txn_amt'].sum().reset_index() 
cc.head()

,cc_no,month,cc_txn_amt
0,2,3,4600
1,2,4,18700
2,2,5,1600
3,2,6,2000
4,4,5,300000


In [218]:
cc = cc.groupby(['cc_no'])['cc_txn_amt'].mean().reset_index() 
cc.head()

,cc_no,cc_txn_amt
0,2,6725.0
1,4,270000.0
2,5,6000.0
3,6,32000.0
4,7,2700.0


In [219]:
kplus['month'] = pd.DatetimeIndex(kplus['sunday']).month
kplus.head()

,id,sunday,kp_txn_count,kp_txn_amt,month
0,14802,2018-01-14,2,2400,1
1,14802,2018-04-01,9,33900,4
2,14802,2018-05-27,7,6100,5
3,14802,2018-01-28,3,8500,1
4,14802,2018-03-25,2,4000,3


In [220]:
kplus = kplus.groupby(['id', 'month'])['kp_txn_amt'].sum().reset_index() 
kplus.head()

,id,month,kp_txn_amt
0,1,6,3500
1,2,1,7400
2,2,2,19600
3,2,3,23500
4,2,4,63400


In [221]:
kplus = kplus.groupby(['id'])['kp_txn_amt'].mean().reset_index() 
kplus.head()

,id,kp_txn_amt
0,1,3.500000e+03
1,2,3.500000e+04
2,3,2.328333e+04
3,4,1.012683e+06
4,6,1.340000e+04


In [222]:
kplus = kplus.rename(columns={"kp_txn_amt": "kp_month_mean"})
kplus.head()

,id,kp_month_mean
0,1,3.500000e+03
1,2,3.500000e+04
2,3,2.328333e+04
3,4,1.012683e+06
4,6,1.340000e+04


In [223]:
cc = cc.rename(columns={"cc_txn_amt": "cc_month_mean"})
cc.head()

,cc_no,cc_month_mean
0,2,6725.0
1,4,270000.0
2,5,6000.0
3,6,32000.0
4,7,2700.0


In [224]:
dmg.head()

,id,cc_no,gender,ocp_cd,age
0,1,1,2,9.0,5
1,1,98397,2,9.0,5
2,2,2,2,3.0,4
3,2,9740,2,3.0,4
4,3,3,2,1.0,3


In [0]:
cid = dmg.merge(cc, how='left', on='cc_no').fillna(0)

In [226]:
cid.head()

,id,cc_no,gender,ocp_cd,age,cc_month_mean
0,1,1,2,9.0,5,0.0
1,1,98397,2,9.0,5,11175.0
2,2,2,2,3.0,4,6725.0
3,2,9740,2,3.0,4,7660.0
4,3,3,2,1.0,3,0.0


In [227]:
cid = cid.groupby(['id'])['cc_month_mean'].sum().reset_index() 
cid.head()

,id,cc_month_mean
0,1,11175.0
1,2,14385.0
2,3,0.0
3,4,500000.0
4,5,6000.0


In [228]:
main = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/main.csv')
main.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cd_ex
0,1,2,9.0,5,20000.0,2.0,3500.0,44700.0
1,2,2,3.0,4,29000.0,104.0,210000.0,65200.0
2,3,2,1.0,3,18000.0,70.0,139700.0,0.0
3,4,2,3.0,5,19000.0,315.0,6076100.0,770000.0
4,5,2,9.0,4,16000.0,0.0,0.0,12000.0


In [229]:
main_full = main.merge(cid, how='left', on='id').fillna(0)
main_full.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cd_ex,cc_month_mean
0,1,2,9.0,5,20000.0,2.0,3500.0,44700.0,11175.0
1,2,2,3.0,4,29000.0,104.0,210000.0,65200.0,14385.0
2,3,2,1.0,3,18000.0,70.0,139700.0,0.0,0.0
3,4,2,3.0,5,19000.0,315.0,6076100.0,770000.0,500000.0
4,5,2,9.0,4,16000.0,0.0,0.0,12000.0,6000.0


In [230]:
main_full = main_full.merge(kplus, how='left', on='id').fillna(0)
main_full.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cd_ex,cc_month_mean,kp_month_mean
0,1,2,9.0,5,20000.0,2.0,3500.0,44700.0,11175.0,3.500000e+03
1,2,2,3.0,4,29000.0,104.0,210000.0,65200.0,14385.0,3.500000e+04
2,3,2,1.0,3,18000.0,70.0,139700.0,0.0,0.0,2.328333e+04
3,4,2,3.0,5,19000.0,315.0,6076100.0,770000.0,500000.0,1.012683e+06
4,5,2,9.0,4,16000.0,0.0,0.0,12000.0,6000.0,0.000000e+00


In [231]:
train_data = main_full[:50000]
test_data = main_full[50000:]
test_data.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cd_ex,cc_month_mean,kp_month_mean
50000,50001,1,9.0,3,0.0,28.0,64600.0,31800.0,10600.000000,10766.666667
50001,50002,1,9.0,4,0.0,131.0,93700.0,5900.0,2950.000000,15616.666667
50002,50003,1,3.0,4,0.0,83.0,202500.0,0.0,0.000000,33750.000000
50003,50004,2,3.0,4,0.0,20.0,11000.0,7200.0,1440.000000,1833.333333
50004,50005,1,3.0,4,0.0,67.0,195500.0,14800.0,4933.333333,32583.333333


In [0]:
train = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/train.csv')

In [233]:
train_data['income'] = train['income']
train_data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cd_ex,cc_month_mean,kp_month_mean
0,1,2,9.0,5,20000,2.0,3500.0,44700.0,11175.0,3.500000e+03
1,2,2,3.0,4,106000,104.0,210000.0,65200.0,14385.0,3.500000e+04
2,3,2,1.0,3,29000,70.0,139700.0,0.0,0.0,2.328333e+04
3,4,2,3.0,5,61000,315.0,6076100.0,770000.0,500000.0,1.012683e+06
4,5,2,9.0,4,18000,0.0,0.0,12000.0,6000.0,0.000000e+00


In [0]:
from google.colab import files

train_data.to_csv('train_data_v4.csv', index=False)
files.download('train_data_v4.csv')



In [0]:
test_data.to_csv('test_data_v4.csv', index=False)
files.download('test_data_v4.csv')